# Inferencing

This notebook is used for inferencing in parallel for many models, saved by the training notebook

At a high level, we use the last 48 hours of meter data (used for lags) and forecasted weather data to predict the load of each
meter using previously trained models.

The historical data required is in this schema (similar to the creation of the training data set):
```json
{
  "MeterNumber": "string",
  "ReadingTimestamp": "datetime",
  "KWHConsumption": "double",
  "Latitude", "double",
  "Longitude", "double",
  "Grouping": "string"
}
```

We then generate a dataset used for the models to use, per meter, that uses the historical values as lag inputs, and the historical weather

In [ ]:

# Data manipulation
# ==============================================================================
import numpy as np
import pandas as pd

# PySpark 
# ==============================================================================
from pyspark.sql.functions import col, min,max,sum,avg, lit, date_trunc, row_number,lag, sum
from pyspark.sql.window import Window

import os
from datetime import datetime
from datetime import timedelta
from datetime import timezone
import requests

import cloudpickle

In [ ]:
#params
lags = [1,2,3,4,22,23,24,46,47,48]

##Enter in Azure Maps Information#####
##don't do this in production, use key vault
########################################
azureMapsClientId = '07e7da45-ab6b-405b-95e5-a64a81f54132'
azureMapsKey = 'pSI8EYMxTkVlM8a1bgqiQ-g1cVVcjHcbUabawH21feo'
#########################################

In [ ]:
# mount the data lake
### 
###  It is up to you on how to mount the data lake container - however, it must be mounted to /mnt/lf for this example
###

Mounted load-forecasting successfully
Out[6]: 'wasbs://load-forecasting@stgmadlssharedcc.blob.core.windows.net'

In [ ]:
#load the historical data here for T - 48 hours
#this is used as a demo. 
latest_meter_reads_df = spark.read.parquet('/mnt/lf/inference/current_readings.parquet')

#read the latest timestamp
latest_timestamp = latest_meter_reads_df\
    .withColumn("ForecastTimestamp", date_trunc('hour',latest_meter_reads_df.ReadingTimestamp))\
    .select(col("ForecastTimestamp"))\
    .orderBy(col("ForecastTimestamp").desc())\
    .toPandas()['ForecastTimestamp'][0]
print(latest_timestamp)

lag_start_time = latest_timestamp + timedelta(hours = -128)
start_pred_time = latest_timestamp
end_pred_time = latest_timestamp + timedelta(hours = 24)

print("Lag start time for prediction: "+str(lag_start_time))
print("Start time for prediction: "+str(start_pred_time))
print("End time for prediction: "+str(end_pred_time))
lag_meter_reads_df = latest_meter_reads_df.where(col("ReadingTimestamp") > lag_start_time)

#convert to hourly
lag_meter_reads_df = lag_meter_reads_df.groupBy("Grouping",date_trunc('hour', lag_meter_reads_df.ReadingTimestamp).alias("ReadingTimestamp"))\
    .agg(\
         avg("Latitude").alias("Latitude"),\
         avg("Longitude").alias("Longitude"),\
         sum("KWHConsumption").alias("KWHConsumption"))

2022-10-28 23:00:00
Lag start time for prediction: 2022-10-23 15:00:00
Start time for prediction: 2022-10-28 23:00:00
End time for prediction: 2022-10-29 23:00:00


## Generate initial inferencing dataset. 

The inferencing dataset is the meter joined with the latest timestamp. 

A high level, we are going to do the following: 

1. Retrieve the latest time for the historical dataset.  This will serve as 'T'
2. Create a dataset that includes a range from T to T+24 (inclusive) for each model.  We will end up with something like:
'MeterNumber','ForecastTimestamp'
3. Generate the lags for our models

In [ ]:
#set up lags
lag_window = Window.partitionBy("Grouping").orderBy(col("ReadingTimestamp"))
select_window = Window.partitionBy("Grouping").orderBy(col("ReadingTimestamp").desc())

inference_start_df = lag_meter_reads_df

# we choose the MAX lags because we need to have all the lags
# to push through when we are predicting. 
for lag_num in range(1, np.amax(lags) + 1):
    lag_column_name = "lag_"+str(lag_num)
    inference_start_df = inference_start_df.withColumn(lag_column_name, lag("KWHConsumption", lag_num).over(lag_window))
    
inference_start_df = inference_start_df.withColumn("row", row_number().over(select_window))\
    .where(col("row") == 1)\
    .drop("row")
display(inference_start_df)

Grouping,ReadingTimestamp,Latitude,Longitude,KWHConsumption,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,lag_8,lag_9,lag_10,lag_11,lag_12,lag_13,lag_14,lag_15,lag_16,lag_17,lag_18,lag_19,lag_20,lag_21,lag_22,lag_23,lag_24,lag_25,lag_26,lag_27,lag_28,lag_29,lag_30,lag_31,lag_32,lag_33,lag_34,lag_35,lag_36,lag_37,lag_38,lag_39,lag_40,lag_41,lag_42,lag_43,lag_44,lag_45,lag_46,lag_47,lag_48
0,2022-10-28T23:00:00.000+0000,51.0,-114.0,17.670000104233623,14.609999869018793,10.719999950379133,12.729999978095293,13.689999943599105,15.540000041946769,15.210000030696392,13.439999919384718,12.680000042542815,12.690000044181943,14.690000057220459,14.620000014081597,10.649999931454659,11.000000014901161,10.190000001341105,10.620000014081597,10.019999964162707,10.640000013634562,11.159999979659915,16.290000109001994,16.100000044330955,16.26999994367361,23.85000003129244,19.909999947994947,15.659999933093786,20.61999991722405,12.679999990388751,13.279999976977706,11.089999973773956,11.38999998755753,14.339999945834279,16.189999906346202,12.370000008493662,14.739999970421195,13.609999937936664,15.189999889582396,10.839999996125698,11.119999960064888,11.900000061839819,13.649999940767884,12.879999991506338,13.33999994955957,13.979999953880906,15.910000059753656,17.750000005587935,19.559999836608768,22.880000174045563,17.94999993033707,17.319999950006604
1,2022-10-28T23:00:00.000+0000,51.0,-114.0,17.039999844506383,14.39000004157424,15.139999940991402,12.040000038221478,11.700000006705523,12.410000011324883,10.239999998360872,8.649999903514981,8.050000032410026,10.219999942928553,8.94999997317791,13.240000078454614,6.509999999776483,6.109999975189567,7.530000006780028,6.579999998211861,8.369999943301082,7.489999990910292,8.680000018328428,12.09999997727573,14.919999873265624,17.849999947473407,21.2300001103431,15.459999926388264,10.779999924823642,13.260000057518482,6.949999963864684,7.659999990835786,13.750000104308128,10.04999995790422,7.949999962002039,9.249999985098839,9.220000011846423,9.210000041872263,13.089999999850988,10.129999900236726,7.7499999944120646,6.46000000834465,7.820000026375055,6.6200000159442425,10.229999946430326,9.289999963715672,7.03000003285706,8.439999965950847,16.429999953135848,20.160000029951334,17.890000076964498,15.959999907761812,11.320000007748604
2,2022-10-28T23:00:00.000+0000,51.0,-114.0,15.529999973252416,17.680000016465783,12.599999949336052,12.869999995455146,18.229999849572778,22.18999987654388,19.630000058561563,19.769999923184514,19.97999998368323,11.960000043734908,12.549999980255961,15.009999997913837,8.709999995306134,8.790000015869737,9.460000019520521,8.959999987855554,10.35999995470047,11.620000002905726,13.780000044032931,17.3600000962615,16.46999995224178,18.849999949336052,20.109999937936664,16.02999997884035,15.969999939203262,18.21999998949468,14.190000044181943,13.999999966472387,18.230000089854002,15.220000080764294,15.230000156909227,17.54000007547438,12.089999984949827,12.890000043436885,17.289999993517995,14.420000087469816,11.049999997019768,11.38999997265637,10.650000043213367,13.089999916031957,12.160000029951334,11.98999996110797,14.000000001862645,19.38999993726611,22.800000086426735,21.85999983549118,22.30999972112477,21.289999971166253,22.420000042766333
3,2022-10-28T23:00:00.000+0000,51.0,-114.0,8.970000023022294,9.470000067725778,8.610000051558018,9.410000024363399,8.09000002220273,10.500000029802322,11.029999980702996,9.430000025779009,9.269999958574772,10.970000006258488,10.509999949485064,10.450000016018748,9.620000014081597,7.63999998010695,8.28999998420477,8.929999899119139,10.38000001758337,11.579999988898635,14.030000073835254,10.77999996393919,17.33000000938773,19.97999992966652,17.49999985843897,15.850000075995922,11.049999922513962,9.770000064745545,11.05000001564622,9.199999997392297,12.150000041350722,10.77000005170703,14.789999973028898,11.120000012218952,10.199999978765845,13.910000002011657,12.699999945238233,10.410000020638108,9.040000040084124,9.50999997369945,9.030

## Determine Weather each Meter

And also determine the weather forecasts for T to T+24 at an hourly interval for those weather stations.

In [ ]:
forecast_times_df = generate_dates(str(start_pred_time), str(end_pred_time))
forecast_times_df = forecast_times_df.withColumnRenamed("date_time_ref","ForecastTimeStamp")

weather_df = inference_start_df.select("Grouping","Latitude","Longitude").crossJoin(forecast_times_df)

#weather_df is used to store the forecast times at a given interval
display(weather_df)

Grouping,Latitude,Longitude,ForecastTimeStamp
0,51.0,-114.0,2022-10-29T01:00:00.000+0000
0,51.0,-114.0,2022-10-29T04:00:00.000+0000
0,51.0,-114.0,2022-10-29T07:00:00.000+0000
0,51.0,-114.0,2022-10-29T10:00:00.000+0000
0,51.0,-114.0,2022-10-29T13:00:00.000+0000
0,51.0,-114.0,2022-10-29T16:00:00.000+0000
0,51.0,-114.0,2022-10-29T23:00:00.000+0000
0,51.0,-114.0,2022-10-29T05:00:00.000+0000
0,51.0,-114.0,2022-10-29T00:00:00.000+0000
0,51.0,-114.0,2022-10-29T08:00:00.000+0000


In [ ]:
import pygeohash as pgh

class ForecastData():
    def __init__(self):   # constructor function using self
        self.GeoHash = None
        self.ForecastTimeStamp = None 
        self.TemperatureC = None 
        self.DewPointC = None 
        self.RelativeHumidity = None 
        self.PrecipitationAmountmm = None 
        self.WindDirectionDegrees = None 
        self.WindSpeedKmh = None
        self.VisibilityKm = None 
        self.StationPressurekPa = None 
        self.Humidex = None 
        self.WindChillC = None 

def load_forecast(geohash, lat, lon):
    forecasts = []
    
    url = f'https://atlas.microsoft.com/weather/forecast/hourly/json?api-version=1.1&query={lat},{lon}&duration=24&subscription-key={azureMapsKey}'
    print(url)
    maps_response = requests.get(url)
    if maps_response.status_code == 200:
        print('forecast loaded successfully!')
        forecast_dict = maps_response.json()
        for forecast_dict in forecast_dict.get('forecasts'):
            forecastDat = ForecastData()
            forecastDat.ForecastTimeStamp = datetime.strptime(forecast_dict.get('date'),'%Y-%m-%dT%H:%M:%S%z')
            #convert to utc
            forecastDat.ForecastTimeStamp = datetime.fromtimestamp(forecastDat.ForecastTimeStamp.timestamp(), tz=timezone.utc)
            
            forecastDat.TemperatureC = forecast_dict.get('temperature').get('value')
            forecastDat.DewPointC = forecast_dict.get('dewPoint').get('value')
            forecastDat.RelativeHumidity = forecast_dict.get('relativeHumidity')
            if forecast_dict.get('totalLiquid') is not None:
                forecastDat.PrecipitationAmountmm = forecast_dict.get('totalLiquid').get('value')
            else:
                forecastDat.PrecipitationAmountmm = 0
                
            if forecast_dict.get('visibility') is not None:
                forecastDat.VisibilityKm = forecast_dict.get('visibility').get('value')
            
            if forecast_dict.get('wind') is not None:
                forecastDat.WindDirectionDegrees = forecast_dict.get('wind').get('direction').get('degrees')
                forecastDat.WindSpeedKmh = forecast_dict.get('wind').get('speed').get('value')
            else:
                forecastDat.WindDirectionDegrees = 0
                forecastDat.WindSpeedKmh = 0
            
            forecastDat.GeoHash = geohash
            forecasts.append(forecastDat)
    else:
        print(f'forecast load failed! Response status code returned: {maps_response.status_code}')
    
    forecasts_pdf = pd.DataFrame([vars(s) for s in forecasts])
    with open('/dbfs/mnt/lf/inference/_forecast_wx_data/'+geohash+'.parquet', mode='wb') as file:
        forecasts_pdf.to_parquet(file)

geohash_udf = udf(lambda lat,lon: pgh.encode(latitude = lat,longitude=lon,precision=7))

In [ ]:
#load each forecast!

weather_geohash_pdf = weather_df.withColumn('GeoHash', geohash_udf('Latitude','Longitude')).select("GeoHash", "Latitude","Longitude").distinct().toPandas()

for i in range(len(weather_geohash_pdf)):
    row = weather_geohash_pdf.iloc[i]
    load_forecast(row.GeoHash,row.Latitude,row.Longitude)

In [ ]:
forecast_data_df = spark.read.parquet('/mnt/lf/inference/_forecast_wx_data/*.parquet')
display(forecast_data_df)

GeoHash,ForecastTimeStamp,TemperatureC,DewPointC,RelativeHumidity,PrecipitationAmountmm,WindDirectionDegrees,WindSpeedKmh,VisibilityKm,StationPressurekPa,Humidex,WindChillC
c3nfju7,2022-11-29T19:00:00.000+0000,-15.6,-23.9,50,0.0,359.0,14.5,16.1,null,null,null
c3nfju7,2022-11-29T20:00:00.000+0000,-15.0,-25.6,42,0.0,358.0,12.9,16.1,null,null,null
c3nfju7,2022-11-29T21:00:00.000+0000,-15.0,-26.1,39,0.0,355.0,9.7,16.1,null,null,null
c3nfju7,2022-11-29T22:00:00.000+0000,-15.6,-27.2,36,0.0,355.0,9.7,16.1,null,null,null
c3nfju7,2022-11-29T23:00:00.000+0000,-16.1,-27.2,39,0.0,24.0,9.7,16.1,null,null,null
c3nfju7,2022-11-30T00:00:00.000+0000,-17.2,-27.2,42,0.0,78.0,9.7,16.1,null,null,null
c3nfju7,2022-11-30T01:00:00.000+0000,-17.8,-27.2,45,0.0,107.0,8.0,16.1,null,null,null
c3nfju7,2022-11-30T02:00:00.000+0000,-18.3,-26.7,47,0.0,114.0,8.0,16.1,null,null,null
c3nfju7,2022-11-30T03:00:00.000+0000,-18.3,-26.7,50,0.0,119.0,8.0,16.1,null,null,null
c3nfju7,2022-11-30T04:00:00.000+0000,-18.9,-26.1,54,0.0,119.0,8.0,16.1,null,null,null


## Build initial training dataset, with forecasting data.

In [ ]:
#this is temporary, because we are assuming the time is *now*
inference_df = inference_start_df.drop("ReadingTimestamp").withColumn("GeoHash", geohash_udf('Latitude','Longitude'))

#usually we would do a JOIN with the weather_df at this point, but we do a  join with the forecast data
inference_df = inference_df.join(forecast_data_df, forecast_data_df.GeoHash == inference_df.GeoHash, 'inner')\
    .drop(inference_df.GeoHash)\
    .drop(forecast_data_df.GeoHash)\
    .orderBy("Grouping","ForecastTimestamp")

inference_df = inference_df.na.fill(value=999)

display(inference_df)

Grouping,Latitude,Longitude,KWHConsumption,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,lag_8,lag_9,lag_10,lag_11,lag_12,lag_13,lag_14,lag_15,lag_16,lag_17,lag_18,lag_19,lag_20,lag_21,lag_22,lag_23,lag_24,lag_25,lag_26,lag_27,lag_28,lag_29,lag_30,lag_31,lag_32,lag_33,lag_34,lag_35,lag_36,lag_37,lag_38,lag_39,lag_40,lag_41,lag_42,lag_43,lag_44,lag_45,lag_46,lag_47,lag_48,ForecastTimeStamp,TemperatureC,DewPointC,RelativeHumidity,PrecipitationAmountmm,WindDirectionDegrees,WindSpeedKmh,VisibilityKm,StationPressurekPa,Humidex,WindChillC
0,51.0,-114.0,17.670000104233623,14.609999869018793,10.719999950379133,12.729999978095293,13.689999943599105,15.540000041946769,15.210000030696392,13.439999919384718,12.680000042542815,12.690000044181943,14.690000057220459,14.620000014081597,10.649999931454659,11.000000014901161,10.190000001341105,10.620000014081597,10.019999964162707,10.640000013634562,11.159999979659915,16.290000109001994,16.100000044330955,16.26999994367361,23.85000003129244,19.909999947994947,15.659999933093786,20.61999991722405,12.679999990388751,13.279999976977706,11.089999973773956,11.38999998755753,14.339999945834279,16.189999906346202,12.370000008493662,14.739999970421195,13.609999937936664,15.189999889582396,10.839999996125698,11.119999960064888,11.900000061839819,13.649999940767884,12.879999991506338,13.33999994955957,13.979999953880906,15.910000059753656,17.750000005587935,19.559999836608768,22.880000174045563,17.94999993033707,17.319999950006604,2022-11-29T19:00:00.000+0000,-15.6,-23.9,50,0.0,359.0,14.5,16.1,999,999,999
0,51.0,-114.0,17.670000104233623,14.609999869018793,10.719999950379133,12.729999978095293,13.689999943599105,15.540000041946769,15.210000030696392,13.439999919384718,12.680000042542815,12.690000044181943,14.690000057220459,14.620000014081597,10.649999931454659,11.000000014901161,10.190000001341105,10.620000014081597,10.019999964162707,10.640000013634562,11.159999979659915,16.290000109001994,16.100000044330955,16.26999994367361,23.85000003129244,19.909999947994947,15.659999933093786,20.61999991722405,12.679999990388751,13.279999976977706,11.089999973773956,11.38999998755753,14.339999945834279,16.189999906346202,12.370000008493662,14.739999970421195,13.609999937936664,15.189999889582396,10.839999996125698,11.119999960064888,11.900000061839819,13.649999940767884,12.879999991506338,13.33999994955957,13.979999953880906,15.910000059753656,17.750000005587935,19.559999836608768,22.880000174045563,17.94999993033707,17.319999950006604,2022-11-29T20:00:00.000+0000,-15.0,-25.6,42,0.0,358.0,12.9,16.1,999,999,999
0,51.0,-114.0,17.670000104233623,14.609999869018793,10.719999950379133,12.729999978095293,13.689999943599105,15.540000041946769,15.210000030696392,13.439999919384718,12.680000042542815,12.690000044181943,14.690000057220459,14.620000014081597,10.649999931454659,11.000000014901161,10.190000001341105,10.620000014081597,10.019999964162707,10.640000013634562,11.159999979659915,16.290000109001994,16.100000044330955,16.26999994367361,23.85000003129244,19.909999947994947,15.659999933093786,20.61999991722405,12.679999990388751,13.279999976977706,11.089999973773956,11.38999998755753,14.339999945834279,16.189999906346202,12.370000008493662,14.739999970421195,13.609999937936664,15.189999889582396,10.839999996125698,11.119999960064888,11.900000061839819,13.649999940767884,12.879999991506338,13.33999994955957,13.979999953880906,15.910000059753656,17.750000005587935,19.559999836608768,22.880000174045563,17.94999993033707,17.319999950006604,2022-11-29T21:00:00.000+0000,-15.0,-26.1,39,0.0,355.0,9.7,16.1,999,999,999
0,51.0,-114.0,17.670000104233623,14.609999869018793,10.719999950379133,12.729999978095293,13.689999943599105,15.540000041946769,15.210000030696392,13.439999919384718,12.680000042542815,12.690000044181943,14.690000057220459,14.620000014081597,10.649999931454659,11.000000014901161,10.190000001341105,10.620000014081597,10.019999964162707,10.640000013634562,11.159999979659915,16.290000109001994,16.100000044330955,16.26999

## Do Inferencing

In [ ]:
def predict_load_for_meter(inference_pdf):
    steps = 24
    target_coumn = 'KWHConsumption'
    
    start = datetime.now()
    #meter_num = inference_pdf["MeterNumber"][0]
    grouping = str(inference_pdf["Grouping"][0])
    
    model_name = 'MeterModel_'+grouping+'.pkl'
    model_dir = '/dbfs/mnt/lf/models/'+grouping
    model_save_path = model_dir + '/' + model_name
    
    print(f'loading model from {model_save_path}')
    
    if not os.path.exists(model_save_path):
        return pd.DataFrame({'group':grouping, 'valid': False, 'time':[None],'predicted_kwh':[None]})
    
    with open(model_save_path, 'rb') as f:
        model = cloudpickle.load(f)
    
    inference_pdf['ForecastTimeStamp'] = pd.to_datetime(inference_pdf['ForecastTimeStamp'])
    inference_pdf = inference_pdf.set_index('ForecastTimeStamp')
    inference_pdf = inference_pdf.sort_index()
    #inference_pdf = inference_pdf.drop(columns='MeterNumber')
    inference_pdf = inference_pdf.drop(columns='Grouping')
    #inference_pdf.drop(columns='Weather') # we don't care about this, future improvement maybe
    
    #generate the time params we are interested in
    inference_pdf['ReadingMonth'] = inference_pdf.index.month
    inference_pdf['ReadingWeekDay'] = inference_pdf.index.day_of_week + 1
    inference_pdf['ReadingHour'] = inference_pdf.index.hour + 1
    inference_pdf['ReadingDay'] = inference_pdf.index.day
    
    columns = ["TemperatureC",\
                "DewPointC",\
                "RelativeHumidity",\
                "PrecipitationAmountmm",\
                "WindDirectionDegrees",\
                "WindSpeedKmh",\
                "VisibilityKm",\
                "StationPressurekPa",\
                "Humidex",\
                "WindChillC",\
                "ReadingMonth",\
                "ReadingWeekDay",\
                "ReadingHour",\
                "ReadingDay"]
  

    for lag in lags:
        columns.append('lag_'+str(lag))
    
    prediction_vect = inference_pdf #our initialization vector. we will use this to make the predictions, and shuffle down the lags
    
    times = []
    preds = []
    
    print("predicting meter load: "+str(grouping))
    for step in range(1,steps):
        prediction_x = prediction_vect[columns].iloc[[step]]  #predict one time at a time
        prediction_result = model.predict(prediction_x)

        prediction_vect['lag_1'] = prediction_result[0]
        #shuffle
        i = np.amax(lags) - 1
        while i > 1:
            prediction_vect['lag_'+str(lag)] = prediction_vect['lag_'+str(lag - 1)]
            i -= 1
        
        times.append(prediction_vect.index[step])
        preds.append(prediction_result[0])
        
    print("completed prediction for meter model: "+str(grouping))
    
    end = datetime.now()
    
    delta = end - start

    # time difference in seconds
    print(f"Model prediction time {grouping}: {delta.total_seconds()} seconds")
        
    return pd.DataFrame({'group':grouping,'valid':True, 'time':times,'predicted_kwh':preds})

In [ ]:
#parallel
inference_df = inference_df.repartition(40)

In [ ]:
inference_result = inference_df\
    .groupBy("Grouping")\
    .applyInPandas(predict_load_for_meter, schema = "group string, valid boolean, time_utc timestamp, predicted_kwh float")

inference_result.write.mode("overwrite").parquet('/mnt/lf/inference/forecast_bygroup.parquet')
display(inference_result)

group,valid,time,predicted_kwh
0,true,2022-11-29T20:00:00.000+0000,15.841278
0,true,2022-11-29T21:00:00.000+0000,17.171062
0,true,2022-11-29T22:00:00.000+0000,17.994022
0,true,2022-11-29T23:00:00.000+0000,18.852146
0,true,2022-11-30T00:00:00.000+0000,19.428883
0,true,2022-11-30T01:00:00.000+0000,19.652073
0,true,2022-11-30T02:00:00.000+0000,19.628502
0,true,2022-11-30T03:00:00.000+0000,19.295357
0,true,2022-11-30T04:00:00.000+0000,19.01281
0,true,2022-11-30T05:00:00.000+0000,18.927084


In [ ]:
#predict_load_for_meter(inference_df.where(col("MeterNumber") == '1095433').toPandas())

## Output Weather
We output the weather we used for forecasting for reporting purposes

In [ ]:
forecast_data_df = inference_df.groupBy("ForecastTimeStamp").agg(\
                                                                 avg("TemperatureC").alias("AvgTemperatureC"),\
                                                                 avg("DewPointC").alias("AvgDewPointC"),\
                                                                 avg("RelativeHumidity").alias("AvgRelativeHumidity"),\
                                                                 avg("PrecipitationAmountmm").alias("AvgPrecipitationAmountmm"),\
                                                                 avg("WindDirectionDegrees").alias("AvgWindDirectionDegrees"),\
                                                                 avg("WindSpeedKmh").alias("AvgWindSpeedKmh"),\
                                                                 avg("VisibilityKm").alias("AvgVisibilityKm"))

forecast_data_df.write.mode("overwrite").parquet('/mnt/lf/inference/weather_forecast_data.parquet')